In [1]:
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import array_to_img, img_to_array, load_img
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import os

In [2]:
original_size_img = (1664, 1128)
half_size_img = (1177, 798)
five_percent_size_img = (372, 252)
two_half_size_img = (263, 178)

## Plotting Precision and Accuracy Function

In [3]:
def plot_loss_and_accuracy(results, final=False):
    
    if final:
        val_label="test"
    else:
        val_label="validation"

    # Extracting metrics from model fitting
    train_loss = results.history['loss']
    val_loss = results.history['val_loss']
    train_precision = results.history['recall']
    val_precision = results.history['val_recall']
    train_accuracy = results.history['binary_accuracy']
    val_accuracy = results.history['val_binary_accuracy']

    # Setting up plots
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(10, 15))

    # Plotting loss info
    ax1.set_title("Loss")
    sns.lineplot(x=results.epoch, y=train_loss, ax=ax1, label="train")
    sns.lineplot(x=results.epoch, y=val_loss, ax=ax1, label=val_label)
    ax1.legend()

    #Plotting recall
    ax2.set_title("Recall")
    sns.lineplot(x=results.epoch, y=train_precision, ax=ax2, label="train")
    sns.lineplot(x=results.epoch, y=val_precision, ax=ax2, label=val_label)
    ax2.legend()
    
    # Plotting accuracy info
    ax3.set_title("Accuracy")
    sns.lineplot(x=results.epoch, y=train_accuracy, ax=ax3, label="train")
    sns.lineplot(x=results.epoch, y=val_accuracy, ax=ax3, label=val_label)
    ax3.legend()

## Image Data Generator

In [4]:
# Directory path
train_data_dir = '../data/localdata/chest_xray/train'
test_data_dir = '../data/localdata/chest_xray/test'

# Get all the data in the directory data/localdata/chest_xray/test (624 images), and reshape them
training_generator = ImageDataGenerator(rescale=1/255, validation_split=0.25)
    #                                     ,
    # rotation_range=30,
    # width_shift_range=0.3,
    # height_shift_range=0.3,
    # horizontal_flip=True)
test_generator = ImageDataGenerator(rescale=1/255)

train_set = training_generator.flow_from_directory(
        train_data_dir, target_size=(300, 300), batch_size=32, color_mode='grayscale', 
        class_mode='binary', seed=42, subset='training')

val_set = training_generator.flow_from_directory(
    train_data_dir, target_size=(300, 300), batch_size=32, color_mode='grayscale', 
         class_mode='binary', seed=42, subset='validation')

test_set = test_generator.flow_from_directory(
        test_data_dir, target_size=(300, 300), batch_size=32, color_mode='grayscale', class_mode='binary')

# Create the datasets
train_images, train_labels = next(train_set)
test_images, test_labels = next(test_set)

Found 6280 images belonging to 2 classes.
Found 2092 images belonging to 2 classes.
Found 1140 images belonging to 2 classes.


## First Complicated Model

In [5]:
# model = models.Sequential()
# model.add(layers.Conv2D(filters=32,
#                        kernel_size=(3, 3),
#                        activation='relu',
#                        input_shape=(200, 200, 1)))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(64, (3, 3), activation='relu', ))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(64, (3, 3), activation='relu', ))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Flatten())
# model.add(layers.Dense(64, activation='relu'))
# model.add(layers.Dense(1, activation='sigmoid'))
# model.compile(optimizer='Adam',
#               loss='BinaryCrossentropy',
#               metrics=['Recall', 'BinaryAccuracy'])

# early_stop = EarlyStopping(monitor='val_binary_accuracy', min_delta=0.001,
#                            verbose=1, patience=4, restore_best_weights=True,
#                            mode='auto')

# results = model.fit(train_set, epochs=50, validation_data=val_set, callbacks=early_stop)

In [6]:
# plot_loss_and_accuracy(results)

In [7]:
# model.evaluate(test_set)

## Hybrid with Lydia Best Model

In [8]:
# best_model = models.Sequential()
# best_model.add(layers.Conv2D(filters=32,
#                         kernel_size=(3, 3),
#                         activation='relu',
#                         input_shape=(250, 250, 1)))
# best_model.add(layers.MaxPooling2D(pool_size=(2, 2)))
# best_model.add(layers.Flatten())
# best_model.add(layers.Dense(64, activation='relu'))
# best_model.add(layers.Dense(1, activation='sigmoid'))

# best_model.compile(optimizer='Adam',
#               loss='BinaryCrossentropy',
#               metrics=['Recall', 'BinaryAccuracy'])

# checkpoint_path = "best_model/cp.ckpt"
# checkpoint_dir = os.path.dirname(checkpoint_path)

# cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
#                                                  save_weights_only=True,
#                                                  verbose=1)

# early_stop = EarlyStopping(monitor='val_binary_accuracy', min_delta=0.001,
#                            verbose=1, patience=4, restore_best_weights=True,
#                            mode='auto')

# results_best = best_model.fit(train_set, epochs=50, validation_data=val_set,
#                     callbacks=[ early_stop])

In [9]:
# plot_loss_and_accuracy(results_best)

In [10]:
# best_model.evaluate(test_set)

## Best Model

In [11]:
model4 = models.Sequential()
model4.add(layers.Conv2D(filters=32,
                        kernel_size=(3, 3),
                        activation='relu',
                        input_shape=(300, 300, 1)))
model4.add(layers.MaxPooling2D(pool_size=(2, 2)))
model4.add(layers.Conv2D(64, (3, 3), activation='relu'))
model4.add(layers.MaxPooling2D((2, 2)))
model4.add(layers.Conv2D(64, (3, 3), activation='relu'))
model4.add(layers.Flatten())
model4.add(layers.Dense(64, activation='relu'))
model4.add(layers.Dense(1, activation='sigmoid'))

model4.compile(optimizer='Adam',
              loss='BinaryCrossentropy',
              metrics=['Recall', 'BinaryAccuracy'])
early_stop = EarlyStopping(monitor='val_binary_accuracy', min_delta=0.001,
                           verbose=1, patience=4, restore_best_weights=True,
                           mode='auto')

results4 = model4.fit(train_set, epochs=50, validation_data=val_set, callbacks=early_stop)
plot_loss_and_accuracy(results4)


Metal device set to: Apple M1


2023-09-13 14:29:46.704652: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-09-13 14:29:46.704819: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/50


2023-09-13 14:29:46.922905: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-09-13 14:29:47.123762: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


 99/197 [==============>...............] - ETA: 39s - loss: 0.3319 - recall: 0.9572 - binary_accuracy: 0.8807

KeyboardInterrupt: 

In [ ]:
model4.evaluate(test_set)